In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "you API key"  
import xml.etree.ElementTree as ET
import requests
import pandas as pd
from datetime import datetime
from openai import OpenAI

class MedicalHistoryAnalyzer:
    def __init__(self, api_key):
        self.medical_llm = OpenAI(api_key=api_key)
        
    def analyze_medical_text(self, prompt):
        """Process medical criteria with AI"""
        try:
            response = self.medical_llm.chat.completions.create(
                model="gpt-3.5-turbo",
                messages=[{"role": "user", "content": prompt}],
                temperature=0.2
            )
            return response.choices[0].message.content.strip()
        except Exception as e:
            print(f"AI analysis error: {e}")
            return ""

class TrialMatchFinder:
    def __init__(self, api_key):
        self.active_trials = []
        self.patient_data = []
        self.ai_helper = MedicalHistoryAnalyzer(api_key)
        
    def fetch_active_trials(self):
        """Get recruiting trials from clinicaltrials.gov"""
        trials_url = "https://clinicaltrials.gov/api/query/full_studies"
        params = {
            "expr": "RECRUITING",
            "min_rnk": 1,
            "max_rnk": 30,
            "fmt": "json"
        }
        
        response = requests.get(trials_url, params=params)
        studies = response.json().get("FullStudiesResponse", {}).get("FullStudies", [])
        
        for study in studies:
            protocol = study["Study"]["ProtocolSection"]
            eligibility = protocol["EligibilityModule"]
            
            self.active_trials.append({
                "trial_id": protocol["IdentificationModule"]["NCTId"],
                "title": protocol["IdentificationModule"].get("OfficialTitle", "Unnamed Trial"),
                "age_range": (
                    int(eligibility["MinimumAge"].split()[0]),
                    int(eligibility["MaximumAge"].split()[0])
                ),
                "gender": eligibility.get("Gender", "All"),
                "conditions": [c.lower() for c in eligibility.get("ConditionList", [])],
                "inclusion": protocol.get("DescriptionModule", {}).get("DetailedDescription", ""),
                "exclusions": eligibility.get("ExclusionCriteria", {}).get("TextBlock", "")
            })

    def process_patient_file(self, xml_path):
        """Read and parse patient XML data"""
        tree = ET.parse(xml_path)
        root = tree.getroot()
        
        for patient in root.findall("Patient"):
            record = {
                "id": patient.find("ID").text,
                "dob": datetime.strptime(patient.find("Demographics/DOB").text, "%Y-%m-%d"),
                "gender": patient.find("Demographics/Gender").text,
                "conditions": self._get_patient_conditions(patient),
                "medications": [d.find("Name").text.lower() 
                              for d in patient.findall("Treatments/CurrentMedications/Drug")],
                "labs": {t.find("Name").text: t.find("Value").text 
                        for t in patient.findall("LabResults/Test")}
            }
            self.patient_data.append(record)
    
    def _get_patient_conditions(self, patient):
        """Extract all medical conditions"""
        conditions = []
        primary = patient.find("MedicalConditions/PrimaryDiagnosis/Condition")
        if primary is not None:
            conditions.append(primary.text.lower())
        
        for cond in patient.findall("MedicalConditions/Comorbidities/Condition"):
            conditions.append(cond.text.lower())
        return conditions

    def calculate_current_age(self, birth_date):
        """Determine patient's age"""
        today = datetime(2024, 2, 15)  # Use current date in real implementation
        return today.year - birth_date.year - ((today.month, today.day) < 
                                              (birth_date.month, birth_date.day))

    def check_eligibility(self, patient, trial):
        """Evaluate match between patient and trial"""
        reasons = []
        age = self.calculate_current_age(patient["dob"])
        
        # Basic checks
        if trial["age_range"][0] <= age <= trial["age_range"][1]:
            reasons.append(f"Age {age} matches range {trial['age_range']}")
        
        if patient["gender"].lower() in trial["gender"].lower():
            reasons.append("Gender matches trial requirements")
        
        # Condition matching
        common_conditions = set(patient["conditions"]) & set(trial["conditions"])
        if common_conditions:
            reasons.append(f"Shared conditions: {', '.join(common_conditions)}")
        
        # AI analysis
        ai_analysis = self.ai_criteria_check(patient, trial)
        if ai_analysis:
            reasons.extend(ai_analysis)
            
        return reasons
    
    def ai_criteria_check(self, patient, trial):
        """Use LLM for complex criteria evaluation"""
        prompt = f"""Analyze patient-trial compatibility:
        
        Patient:
        - Age: {self.calculate_current_age(patient['dob'])}
        - Gender: {patient['gender']}
        - Conditions: {', '.join(patient['conditions'])}
        - Medications: {', '.join(patient['medications'])}
        - Lab Results: {patient['labs']}
        
        Trial Requirements:
        {trial['inclusion']}
        
        Exclusions:
        {trial['exclusions']}
        
        Output format:
        Matches: [list]
        Conflicts: [list]"""
        
        response = self.ai_helper.analyze_medical_text(prompt)
        return self._parse_ai_response(response)

    def _parse_ai_response(self, text):
        """Convert LLM output to structured format"""
        try:
            matches_line = [line for line in text.split("\n") if "Matches:" in line][0]
            conflicts_line = [line for line in text.split("\n") if "Conflicts:" in line][0]
            
            matches = matches_line.split(":")[1].strip().split(", ")
            conflicts = conflicts_line.split(":")[1].strip().split(", ")
            
            results = []
            if matches and matches[0] != "None":
                results.append(f"AI Matches: {', '.join(matches)}")
            if conflicts and conflicts[0] != "None":
                results.append(f"AI Warnings: {', '.join(conflicts)}")
                
            return results
        except:
            return ["AI analysis incomplete"]

    def run_matching(self, input_xml, output_file):
        """Main execution flow"""
        self.fetch_active_trials()
        self.process_patient_file(input_xml)
        
        results = []
        for patient in self.patient_data:
            matches = []
            for trial in self.active_trials:
                eligibility = self.check_eligibility(patient, trial)
                if eligibility:
                    matches.append({
                        "trialId": trial["trial_id"],
                        "trialName": trial["title"],
                        "eligibilityReasons": eligibility
                    })
            
            results.append({
                "patientId": patient["id"],
                "eligibleTrials": matches
            })
        
        pd.DataFrame(results).to_excel(output_file, index=False)
        print(f"Success! Results saved to {output_file}")
        return results

if __name__ == "__main__":
    matcher = TrialMatchFinder(os.getenv("OPENAI_API_KEY"))
    
    matcher.run_matching(
        input_xml="add input path as xml file",
        output_file="add output path as xlsv file"
    )